In [1]:
import ollama 
import os
from tqdm import tqdm
import torch
import signal
import random
import numpy as np
import json
import numpy as np
from PIL import Image
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import AutoProcessor, CLIPModel # pip install transformers
# jupyter notebook --ip 0.0.0.0 --port 8889 --allow-root
# kubectl port-forward dep-cvpr-7d4d4b94f9-82829 8889:8889

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The

In [2]:
base_path = '/root/home/data/hateful_memes/'
images_path = os.path.join(base_path, "img")

list_of_image_names = os.listdir(images_path)

In [3]:
def check_yes_no(text):
    # Strip any leading/trailing whitespace and convert to lowercase
    text = text.strip().lower()

    # Check if the text starts with 'yes' or 'no'
    if text.startswith("yes"):
        return 1
    elif text.startswith("no"):
        return 0
    else:
        return None  
    
class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException


def read_jsonl_file(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            entry = json.loads(line.strip())
            data.append(entry)
    return data

def load_image_and_label(entry, img_base_path):
    img_path = f"{img_base_path}/{entry['img']}"
    try:
        img = Image.open(img_path)
        label = entry['label']
        return img, label
    except FileNotFoundError:
        print(f"Image {img_path} not found.")
        return None, None


def load_dev_file(input_file):
    dev_data = {}
    with open(input_file, 'r') as infile:
        for line in infile:
            # Load each JSON line as a dictionary
            entry = json.loads(line.strip())
        
            # Use the image path as the key and the label as the value
            dev_data[entry["img"]] = entry["label"]
    return dev_data

dev_file = os.path.join(base_path, "dev.jsonl")
dev_data = load_dev_file(dev_file)

In [4]:
list_of_models = ['llava:7b', 
                  'llava:13b',
                  'llava:34b',
                  'llava-llama3',
                  'bakllava',
                  'moondream',
                  'minicpm-v',
                  'llava-phi3']

ollama.pull('llava:7b') #pull the desired model

{'status': 'success'}

In [ ]:
# Split dev_file
hateful_data = {}
nonhateful_data = {}

for key in dev_data :
    if dev_data[key] == 1 :
        hateful_data[key] = dev_data[key]
    else  :
        nonhateful_data[key] = dev_data[key]
hateful_images = list(hateful_data.keys())
nonhateful_images = list(nonhateful_data.keys())


# Prepare CLIP Model

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
device = "cuda"
model.to(device)

In [5]:
# emb1 = hateful_embeddings['img/08291.png']
# emb1_name = 'img/08291.png'
#print(len(hateful_embeddings))
emb_path='/mnt/Software/ViGIR_CVPR_LLM/prompting_framework/hateful_memes_embeddings'
#torch.save(hateful_embeddings, os.path.join(emb_path,'hateful_embeddings.pth'))
#torch.save(nonhateful_embeddings, os.path.join(emb_path,'nonhateful_embeddings.pth'))
lhe = torch.load(os.path.join(emb_path,'hateful_embeddings.pth'))
lnhe = torch.load(os.path.join(emb_path,'nonhateful_embeddings.pth'))
all_embeddings = lhe | lnhe
# print(len(all_embeddings))
def retrieve_similar(query_embedding, query_image_name, embeddings_dict):
    max_cosine_similarity = -float('inf')
    closest_image=None
    keys = list(embeddings_dict.keys())
    for key in keys:
        if query_image_name != key :
            current_embedding = embeddings_dict[key]
            # Normalize embeddings
            current_embedding = current_embedding/current_embedding.norm(p=2,dim=-1,keepdim=True)
            query_embedding = query_embedding/query_embedding.norm(p=2,dim=-1,keepdim=True)
            
            cosine_similarity = torch.nn.functional.cosine_similarity(current_embedding, query_embedding, dim=1)
            #print(cosine_similarity)
            if cosine_similarity > max_cosine_similarity :
                max_cosine_similarity = cosine_similarity
                closest_image = key
                #print(f"new max, {closest_image}, {cosine_similarity}")
        
    
    return closest_image
# print(x)
#x = retrieve_similar(emb1, emb1_name, lnhe)

In [ ]:
hateful_embeddings['img/08291.png'].shape
print(len(hateful_embeddings))
emb_path='/mnt/Software/ViGIR_CVPR_LLM/prompting_framework/hateful_memes_embeddings'
#torch.save(hateful_embeddings, os.path.join(emb_path,'hateful_embeddings.pth'))
#torch.save(nonhateful_embeddings, os.path.join(emb_path,'nonhateful_embeddings.pth'))
lhe = torch.load(os.path.join(emb_path,'hateful_embeddings.pth'))
lnhe = torch.load(os.path.join(emb_path,'nonhateful_embeddings.pth'))


In [ ]:
for 

In [ ]:
hateful_embeddings = {}
nonhateful_embeddings = {}
for image in tqdm(hateful_images) :
    img_file = os.path.join(base_path, image)
    print(f"Processing Hateful: {img_file}")
    img = Image.open(img_file)
    inputs = processor(images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
    # Move embeddings to CPU
    hateful_embeddings[image]=image_features.cpu()
    del inputs

for image in tqdm(nonhateful_images) :
    img_file = os.path.join(base_path, image)
    print(f"Processing Non-Hateful: {img_file}")
    img = Image.open(img_file) 
    inputs = processor(images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
    nonhateful_embeddings[image]=image_features.cpu()
    del inputs

In [ ]:
options= {  # new
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048, # must be set, otherwise slightly random output
        }
    
llava_7b_emb_labels = {}

np.random.seed(0)

timeout_duration = 20 # 20 seconds
print(f"Handling the timeout exceptions with timeout duration of {timeout_duration} seconds")

#for image_name in tqdm(list_of_image_names):
for key, value in dev_data.items():
    print(f"Image: {key}, Label: {value}")
    # Extract current image name
    query_img = key
    # Extract its embedding 
    query_emb = all_embeddings[key]

    # Calculate its "nearest neighbor" in hateful and non-hateful -- Ramy
    similar_hateful = retrieve_similar(query_emb, query_img, lhe)
    similar_nonhateful = retrieve_similar(query_emb, query_img, lnhe)
    
    #random_image_index = np.random.randint(0, len(dev_data))
    #random_image_path = os.path.join(base_path, images_list[random_image_index])
    
    #image_path = os.path.join(base_path, key)  
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout_duration)  # Set the timeout
    
    prompt_1 = "This is  an offensive meme."
    prompt_2 = "This is not an offensive meme."
    prompt_3 = "How about this one? (Answer only Yes or No)"
    
    try:
        response_1 = ollama.generate(model='llava:7b', prompt=prompt_1, images=[os.path.join(base_path,similar_hateful)], options=options)
        response_2 = ollama.generate(model='llava:7b', prompt=prompt_2, images=[os.path.join(base_path,similar_nonhateful)], options=options, context=response_1['context'])
        response_3 = ollama.generate(model='llava:7b', prompt=prompt_3, images=[os.path.join(base_path,query_img)], options=options, context=response_2['context'])
        label = check_yes_no(response_3['response'])
    except TimeoutException:
        print(f"Prompt for {image_name} took longer than {timeout_duration} seconds. Moving to the next one.")
        label = None

    finally:
        signal.alarm(0)  # Disable the alarm

    llava_7b_emb_labels[query_img] = label
    
    print(f"model results -- Image: {query_img}, Label: {label}")
    print("------------------------------------------------------")
   

# Run prompting with image embeddings from CLIP

In [6]:


class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException

# Setup options and random seed
options = {
    "seed": 123,
    "temperature": 0,
    "num_ctx": 2048,
}
np.random.seed(0)
timeout_duration = 20  # 20 seconds

# Initialize the labels dictionary
llava_7b_emb_labels = {}

# Assign the signal handler for the timeout
signal.signal(signal.SIGALRM, timeout_handler)
print(f"Handling the timeout exceptions with a timeout duration of {timeout_duration} seconds")

for query_img, value in dev_data.items():
    print(f"Image: {query_img}, Label: {value}")

    # Extract embedding
    query_emb = all_embeddings[query_img]

    # Find nearest neighbors
    similar_hateful = retrieve_similar(query_emb, query_img, lhe)
    similar_nonhateful = retrieve_similar(query_emb, query_img, lnhe)

    # Prompts
    prompt_1 = "This is an offensive meme."
    prompt_2 = "This is not an offensive meme."
    prompt_3 = "How about this one? (Answer only Yes or No)"

    # Set the alarm for timeout
    signal.alarm(timeout_duration)
    try:
        response_1 = ollama.generate(
            model='llava:7b', prompt=prompt_1, images=[os.path.join(base_path, similar_hateful)], options=options
        )
        response_2 = ollama.generate(
            model='llava:7b', prompt=prompt_2, images=[os.path.join(base_path, similar_nonhateful)], options=options, context=response_1['context']
        )
        response_3 = ollama.generate(
            model='llava:7b', prompt=prompt_3, images=[os.path.join(base_path, query_img)], options=options, context=response_2['context']
        )
        label = check_yes_no(response_3['response'])
    except TimeoutException:
        print(f"Prompt for {query_img} took longer than {timeout_duration} seconds. Moving to the next one.")
        label = None
    finally:
        signal.alarm(0)  # Disable the alarm

    # Store the result
    llava_7b_emb_labels[query_img] = label
    
    print(f"Model results -- Image: {query_img}, Label: {label}")
    print("------------------------------------------------------")


Handling the timeout exceptions with a timeout duration of 20 seconds
Image: img/08291.png, Label: 1
Model results -- Image: img/08291.png, Label: 0
------------------------------------------------------
Image: img/46971.png, Label: 1
Model results -- Image: img/46971.png, Label: 0
------------------------------------------------------
Image: img/03745.png, Label: 1
Model results -- Image: img/03745.png, Label: 1
------------------------------------------------------
Image: img/83745.png, Label: 1
Model results -- Image: img/83745.png, Label: 0
------------------------------------------------------
Image: img/80243.png, Label: 1
Model results -- Image: img/80243.png, Label: 0
------------------------------------------------------
Image: img/05279.png, Label: 1
Model results -- Image: img/05279.png, Label: 0
------------------------------------------------------
Image: img/01796.png, Label: 1
Model results -- Image: img/01796.png, Label: 1
-----------------------------------------------

KeyError: 'context'

In [ ]:
# Experiment 3 | two prompts

options= {  # new
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048, # must be set, otherwise slightly random output
        }
    
labels_2context_dict = {}

np.random.seed(0)
for model in list_of_models :
    for entry in tqdm(dev_data):
        rand_hateful_index = np.random.randint(0, len(gt_hateful))
        rand_not_hateful_index = np.random.randint(0, len(gt_not_hateful))
        
        hateful_path = os.path.join(base_path, "img", gt_hateful[rand_hateful_index])
        not_hateful_path = os.path.join(base_path, "img", gt_not_hateful[rand_not_hateful_index])
        
        image_path = os.path.join(base_path, entry['img'])
        
        prompt_1 = "This is a offensive meme."
        prompt_2 = "This is not a offensive meme. "
        prompt_3 = "Based on the two previous prompts. Is this an offensive meme? answer either yes or no?. "
    
        
        response_1 = ollama.generate(model=model, prompt=prompt_1, images=[hateful_path], options=options)
        response_2 = ollama.generate(model=model, prompt=prompt_2, images=[not_hateful_path], options=options, context=response_1['context'])
        response_3 = ollama.generate(model=model, prompt=prompt_3, images=[image_path], options=options, context=response_2['context'])
    
        label_2context = check_yes_no(response_3['response'])
    
        image_name =  os.path.basename(entry['img'])
        print(label_2context, '--', entry['label'], '--', entry['img'], '--', image_name)
    
        labels_2context_dict[image_name] = label_2context
        break

In [ ]:

def compute_metrics(gt, predictions, name, output_file="metrics.csv"):
    # Ensure we only evaluate on common keys
    common_keys = set(gt.keys()).intersection(predictions.keys())
    
    # Extract lists of labels based on the common keys, filtering out None values
    y_true = []
    y_pred = []
    for key in common_keys:
        pred_label = predictions[key]
        if pred_label is not None:
            y_true.append(gt[key])
            y_pred.append(pred_label)
    
    # Check if there are valid entries left after filtering
    if y_true and y_pred:
        # Calculate metrics
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        accuracy = accuracy_score(y_true, y_pred)
        
        # Create a DataFrame to store the results
        metrics_df = pd.DataFrame({
            "Model": [name],
            "Precision": [precision],
            "Recall": [recall],
            "F1 Score": [f1],
            "Accuracy": [accuracy]
        })
        
        # Display the table
        print(metrics_df)
        
        # Save to a file (append if file already exists)
        with open(output_file, "a") as f:
            metrics_df.to_csv(f, index=False, header=f.tell()==0)
    else:
        print(f"No valid entries to compute metrics for {name}")

# Example usage
# compute_metrics(gt_dict, predictions_dict, 'Model Metrics')
